# ライブラリのインストール
必要なライブラリをインストールします。後で使うものもまとめてimportします。

```
pip install pandas datasets mecab-pytyon3 unidic-lite
```

# ライブラリのインストール
必要なライブラリをimportします。後で使うものもまとめてimportします。

In [172]:
import pandas as pd
import MeCab
import copy
import tqdm
from typing import Dict, List, Any, Tuple, Optional, Callable
import datasets

# 解析用クラスの作成
文章を発音に変換する関数や単語境界をまたぐ有無などを判定する関数などをひとまとめにしたクラスを作ります。

In [168]:

class SerendipitousWordDetector:
    def __init__(self, tokenize_func: Optional[Callable[[str], Tuple[List[str], List[str]]]] = None):
        """
        SerendipitousWordDetectorクラスのコンストラクタです。
        形態素解析を行うためのトークナイズ関数を受け取り、インスタンス変数に設定します。
        もしトークナイズ関数が指定されていない場合は、デフォルトのトークナイズ関数を使用します。

        Args:
            tokenize_func (Optional[Callable[[str], Tuple[List[str], List[str]]]], optional): 
                形態素解析を行うための関数。
                この関数はテキストを引数に取り、表層形と発音のトークンのリストをタプルで返す必要があります。
                もしNoneが指定された場合は、MeCabを使用したデフォルトのトークナイズ関数が使用されます。
        """
        if callable(tokenize_func):
            self.tokenize = tokenize_func
        else:
            self.tokenize = self.get_default_tokenize_func()
                            
    def get_pronunciation(self, text: str) -> str:
        """
        与えられたテキストを形態素解析し、その発音を連結した文字列を返します。

        Args:
            text (str): 発音を取得したい日本語のテキスト。

        Returns:
            str: テキストの発音を表す文字列。
        """
        _, pronunciation_tokens = self.tokenize(text)
        return "".join(pronunciation_tokens)    
    def get_default_tokenize_func(self):

        mecab = MeCab.Tagger()

        def _tokenize(text: str) -> Tuple[List[str], List[str]]:
            """
            与えられたテキストを形態素解析し、表層形と発音のトークンのリストを返します。

            Args:
                text (str): 形態素解析を行いたい日本語のテキスト。

            Returns:
                Tuple[List[str], List[str]]: 
                    - 最初のリストはテキストの表層形のトークンを含みます。
                    - 二番目のリストは対応する発音のトークンを含みます。
                    これらのリストは同じ長さで、各表層形のトークンは対応する発音のトークンと位置を合わせています。
            """
            lines = mecab.parse(text).splitlines()[:-1]  # EOSを除外するために最後の行を除く
            surface_tokens = [line.split("\t")[0] for line in lines]  # 各行の表層形を抽出
            pronunciation_tokens = [line.split("\t")[1] for line in lines]  # 各行の発音を抽出

            return surface_tokens, pronunciation_tokens

        return _tokenize
    
    def is_word_used_in_original_context(
            self,
            word_surface: str,
            word_pronunciation: str,
            passage_surface: str,
            passage_pronunciation_tokens: List[str]
    ) -> bool:
        """
        単語が元の文脈（意味）で使用されているかどうかを判定します。

        この関数は、単語の表層形が文章中に存在するか、または単語の発音が文章の発音トークンリストに含まれているかどうかをチェックします。
        どちらか一方が真であれば、その単語は元の文脈で使用されていると判定されます。

        Args:
            word_surface (str): チェックしたい単語の表層形。
            word_pronunciation (str): チェックしたい単語の発音。
            passage_surface (str): チェック対象の文章。
            passage_pronunciation_tokens (List[str]): 文章の発音をトークン化したリスト。

        Returns:
            bool: 単語が元の文脈で使用されている場合はTrue、そうでない場合はFalse。
        """
        return (
            word_surface in passage_surface or
            word_pronunciation in passage_pronunciation_tokens
        )    

    def is_crossing_word_boundary(
            self,
            word_surface: str,
            word_pronunciation: str,
            passage_surface: str,
            passage_pronunciation_tokens: List[str]
    ) -> bool:
        """
        単語が文章の単語境界をまたいでいるかどうかを判定します。

        この関数は、単語の発音が文章の発音トークン列に含まれているか（contains_word）、
        単語が元の文脈で使用されていないか（is_word_used_in_original_context）、
        そして単語の発音が単語境界をまたがない形で文章の発音トークン列に含まれているか（is_not_crossing_word_boundary）
        をチェックします。

        Args:
            word_surface (str): チェックしたい単語の表層形。
            word_pronunciation (str): チェックしたい単語の発音。
            passage_surface (str): チェック対象の文章の表層形。
            passage_pronunciation_tokens (List[str]): 文章の発音をトークン化したリスト。

        Returns:
            bool: 単語が単語境界をまたいでいる場合はTrue、そうでない場合はFalse。
        """
        # 単語の発音が文章の発音トークン列に含まれているかどうか
        contains_word = word_pronunciation in "".join(passage_pronunciation_tokens)
        
        # 単語が元の文脈で使用されているかどうか
        is_word_used_in_original_context = self.is_word_used_in_original_context(
            word_surface, word_pronunciation, passage_surface, passage_pronunciation_tokens
        )
        
        # 単語の発音が単語境界をまたがない形で文章の発音トークン列に含まれているかどうか
        is_not_crossing_word_boundary = word_pronunciation in " ".join(passage_pronunciation_tokens)

        # すべての条件を満たす場合、単語は単語境界をまたいでいると判定
        return contains_word and not is_word_used_in_original_context and not is_not_crossing_word_boundary
    
    def get_word_context(
            self,
            word_pronunciation: str,
            passage_surface_tokens: List[str],
            passage_pronunciation_tokens: List[str],
            context_range: Tuple[int] = [10, 10]
    )->str:
        """
        指定された単語の発音に基づいて、その単語が含まれる文章の一部分を抽出します。
        文章は表層形のトークンのリストと発音のトークンのリストで表され、
        単語の発音が文章中で最初に現れる位置を基準に前後のトークンを含めた範囲を返します。

        Args:
            word_pronunciation (str): 抽出したい単語の発音。
            passage_surface_tokens (List[str]): 文章の表層形のトークンのリスト。
            passage_pronunciation_tokens (List[str]): 文章の発音のトークンのリスト。
            context_range (int, optional): 抽出する範囲のトークン数。デフォルトは20。

        Returns:
            str: 指定された単語を含む文章の一部分。
        """

        # 単語の発音が文章の発音トークン列で最初に現れる位置を見つける
        pronunciation_pos = "".join(passage_pronunciation_tokens).index(word_pronunciation)

        # 単語が見つかったトークンのインデックスを特定する
        hit_index_start, hit_index_end = -1, -1
        current_pos = 0
        for i, p in enumerate(passage_pronunciation_tokens):
            current_pos += len(p)
            # 現在のトークンの終わりが単語の発音の開始位置を超えたらループを終了
            if current_pos > pronunciation_pos and hit_index_start < 0:
                hit_index_start = i
            if current_pos >= pronunciation_pos + len(word_pronunciation):
                hit_index_end = i + 1
                break

        # 抽出する範囲の開始インデックスを計算する
        start_index = max(0, hit_index_start-context_range[0])
        # 抽出する範囲の終了インデックスを計算する
        end_index = min(hit_index_end+context_range[1], len(passage_surface_tokens))

        # 指定された範囲の表層形トークンを結合して返す
        return "".join(passage_surface_tokens[start_index: end_index])
    
    def check_hit(self, word_surface: str, word_pronunciation: str
                  , passage_surface_tokens: List[str], passage_pronunciation_tokens: List[str]
                  )->Dict[str, Any]:
        """
        単語が文章中にヒットするかどうかをチェックし、ヒットした場合にはその単語の情報を含む辞書を返します。

        Args:
            word_surface (str): チェックしたい単語の表層形。
            word_pronunciation (str): チェックしたい単語の発音。
            passage_surface_tokens (List[str]): 文章の表層形のトークンのリスト。
            passage_pronunciation_tokens (List[str]): 文章の発音のトークンのリスト。

        Returns:
            Dict[str, Any]: 単語のヒット情報を含む辞書。ヒットしたかどうか、単語の表層形、発音、長さ、
                             単語境界をまたいでいるかどうか、コンテキスト、マッチした文章の表層形を含む。
        """
        # 単語が文章の発音トークン列に含まれているかどうか、および元のコンテキストで使用されていないかどうかをチェック
        hit = word_pronunciation in "".join(passage_pronunciation_tokens) and not self.is_word_used_in_original_context(word_surface, word_pronunciation, "".join(passage_surface_tokens), passage_pronunciation_tokens)
        # 結果を格納する辞書を初期化
        result = {
            "hit": hit,
            "surface": word_surface,
            "pronunciation": word_pronunciation,
            "length": len(word_pronunciation)
        }
        # ヒットした場合、追加の情報を辞書に追加
        if hit:
            # 単語が単語境界をまたいでいるかどうかをチェック
            is_crossing_boundary = self.is_crossing_word_boundary(word_surface, word_pronunciation, "".join(passage_pronunciation_tokens), passage_pronunciation_tokens)
            # 単語のコンテキストを取得
            context = self.get_word_context(word_pronunciation, passage_surface_tokens, passage_pronunciation_tokens)
            if is_crossing_boundary:
                matched_passage_surface = self.get_word_context(word_pronunciation, passage_surface_tokens, passage_pronunciation_tokens, [0,0])
            else:
                matched_passage_surface = self.get_word_context(word_pronunciation, passage_surface_tokens, passage_pronunciation_tokens, [0,0])
            result.update({
                "is_crossing_word_boundary": is_crossing_boundary,
                "context": context,
                "matched_passage_surface": matched_passage_surface
            })
        
        return result


        
    def check_hits(self, word_df: pd.DataFrame, passage_df: pd.DataFrame
                   )->pd.DataFrame:
        """
        単語リストと文章リストを受け取り、各文章に含まれる単語のヒット情報をチェックします。

        Args:
            word_df (pd.DataFrame): チェックしたい単語のデータフレーム。'surface'と'pronunciation'の列が必要です。
            passage_df (pd.DataFrame): チェック対象の文章のデータフレーム。'surface'列が必要です。

        Returns:
            pd.DataFrame: 各文章にヒットした単語の情報を含む新しいデータフレーム。
        """
        # データフレームのディープコピーを作成
        word_df = copy.deepcopy(word_df)
        passage_df = copy.deepcopy(passage_df)

        # 単語データフレームに発音列がない場合は生成
        if "pronunciation" not in word_df.columns:
            word_df["pronunciation"] = word_df["surface"].map(self.get_pronunciation)
        # 文章データフレームに発音トークン列がない場合は生成
        if "pronunciation_tokens" not in passage_df.columns:
            passage_df["surface_tokens"] = passage_df["surface"].map(lambda x: self.tokenize(x)[0])
            passage_df["pronunciation_tokens"] = passage_df["surface"].map(lambda x: self.tokenize(x)[1])
            passage_df["pronunciation"] = passage_df["pronunciation_tokens"].map(lambda x: "".join(x))

        # ヒットした単語を格納するリストを初期化
        hitwords_list = []
        # 各文章に対してヒットした単語をチェック
        for passage_surface_tokens, passage_pronunciation_tokens in tqdm.tqdm(zip(passage_df["surface_tokens"], passage_df["pronunciation_tokens"])):
            hitwords = []
            # 各単語に対してヒットチェックを行い、ヒットしたものをリストに追加
            for word_surface, word_pronunciation in zip(word_df["surface"], word_df["pronunciation"]):
                result = self.check_hit(word_surface, word_pronunciation
                                   , passage_surface_tokens, passage_pronunciation_tokens)
                if result["hit"]:
                    hitwords.append(result)
            
            # ヒットした単語の情報を含むオブジェクトを作成
            hitwords_obj = {"hitwords": hitwords, "hit": False}
            if hitwords:
                hitwords_obj.update({
                    "hit": True
                    , "sum_length": sum([obj["length"] for obj in hitwords])
                    , "max_length": max([obj["length"] for obj in hitwords])
                    , "num": len(hitwords)
                })
            hitwords_list.append(hitwords_obj)
        
        # ヒット情報を文章データフレームに追加
        passage_df["hitword"] = hitwords_list
        # ヒットした単語がある文章のみをフィルタリング
        passage_df = passage_df[passage_df["hitword"].map(lambda x: x["hit"])]
        return passage_df

    def convert_table(self, passage_df: pd.DataFrame) -> pd.DataFrame:
        """
        passage_dfから新しいデータフレームを作成し、各行にヒットした単語の情報を含む表を生成します。

        Args:
            passage_df (pd.DataFrame): 'curid', 'title', および 'hitword' の列を含むデータフレーム。
                'hitword' 列は、ヒットした単語の情報を含む辞書のリストを持つ必要があります。

        Returns:
            pd.DataFrame: ヒットした単語の情報を含む新しいデータフレーム。
        """
        new_rows = []
        for idx, row in tqdm.tqdm(passage_df.iterrows()):
            curid, title = row["curid"], row["title"]
            hitwords_obj = row["hitword"]
            hitwords = hitwords_obj["hitwords"]
            sum_length = hitwords_obj["sum_length"]
            max_length = hitwords_obj["max_length"]
            num = hitwords_obj["num"]
            for hitword in hitwords:
                word_surface, word_pronunciation = hitword["surface"], hitword["pronunciation"]
                word_length = hitword["length"]
                is_crossing_word_boundary = hitword["is_crossing_word_boundary"]
                context = hitword["context"]
                matched_passage_surface = hitword["matched_passage_surface"]

                new_rows.append({
                    "curid": curid,
                    "title": title,
                    "word_surface": word_surface,
                    "word_pronunciation": word_pronunciation,
                    "word_length": word_length,
                    "is_crossing_word_boundary": is_crossing_word_boundary,
                    "context": context,
                    "matched_passage_surface": matched_passage_surface,
                    "sum_length": sum_length,
                    "max_length": max_length,
                    "num": num
                })
        # 新しいデータフレームを作成
        df = pd.DataFrame(new_rows)
        # 重複する行を削除
        df = df.drop_duplicates(subset=['word_surface', 'matched_passage_surface'], keep='first')
        return df

# テスト
各メソッドの出力を確認してみます。

In [174]:
# SerendipitousWordDetectorクラスの定義は省略（上記で提供されたクラス定義を使用）

# テスト用のテキストと単語リストを作成
test_text = "これはテストの文章です。形態素解析を行います。Pythonはプログラミング言語の一つです。"
test_words = ['テスト', '解析', 'Python', 'グゲ']

# SerendipitousWordDetectorクラスのインスタンスを作成
detector = SerendipitousWordDetector()

# tokenizeメソッドのテスト
print("tokenizeメソッドの出力:")
print(detector.tokenize(test_text))

# get_pronunciationメソッドのテスト
print("\nget_pronunciationメソッドの出力:")
print(detector.get_pronunciation(test_text))

# is_word_used_in_original_contextメソッドのテスト
print("\nis_word_used_in_original_contextメソッドの出力:")
surface_tokens, pronunciation_tokens = detector.tokenize(test_text)
for word in test_words:
    print(f"単語 '{word}' が元の文脈で使用されているか: {detector.is_word_used_in_original_context(word, detector.get_pronunciation(word), test_text, pronunciation_tokens)}")

# is_crossing_word_boundaryメソッドのテスト
print("\nis_crossing_word_boundaryメソッドの出力:")
for word in test_words:
    print(f"単語 '{word}' が単語境界をまたいでいるか: {detector.is_crossing_word_boundary(word, detector.get_pronunciation(word), test_text, pronunciation_tokens)}")

# get_word_contextメソッドのテスト
print("\nget_word_contextメソッドの出力:")
for word in test_words:
    print(f"単語 '{word}' のコンテキスト: {detector.get_word_context(detector.get_pronunciation(word), surface_tokens, pronunciation_tokens)}")

# check_hitメソッドのテスト
print("\ncheck_hitメソッドの出力:")
for word in test_words:
    result = detector.check_hit(word, detector.get_pronunciation(word), surface_tokens, pronunciation_tokens)
    print(f"単語 '{word}' のヒット情報: {result}")

# check_hitsメソッドのテスト
print("\ncheck_hitsメソッドの出力:")
word_df = pd.DataFrame({'surface': test_words})
passage_df = pd.DataFrame({'surface': [test_text]})
hitwords_df = detector.check_hits(word_df, passage_df)
print(hitwords_df)

tokenizeメソッドの出力:
(['これ', 'は', 'テスト', 'の', '文章', 'です', '。', '形態', '素', '解析', 'を', '行い', 'ます', '。', 'Python', 'は', 'プログラミング', '言語', 'の', '一', 'つ', 'です', '。'], ['コレ', 'ワ', 'テスト', 'ノ', 'ブンショー', 'デス', '', 'ケータイ', 'ソ', 'カイセキ', 'オ', 'オコナイ', 'マス', '', 'Python', 'ワ', 'プログラミング', 'ゲンゴ', 'ノ', 'ヒト', 'ツ', 'デス', ''])

get_pronunciationメソッドの出力:
コレワテストノブンショーデスケータイソカイセキオオコナイマスPythonワプログラミングゲンゴノヒトツデス

is_word_used_in_original_contextメソッドの出力:
単語 'テスト' が元の文脈で使用されているか: True
単語 '解析' が元の文脈で使用されているか: True
単語 'Python' が元の文脈で使用されているか: True
単語 'グゲ' が元の文脈で使用されているか: False

is_crossing_word_boundaryメソッドの出力:
単語 'テスト' が単語境界をまたいでいるか: False
単語 '解析' が単語境界をまたいでいるか: False
単語 'Python' が単語境界をまたいでいるか: False
単語 'グゲ' が単語境界をまたいでいるか: True

get_word_contextメソッドの出力:
単語 'テスト' のコンテキスト: これはテストの文章です。形態素解析を行います
単語 '解析' のコンテキスト: これはテストの文章です。形態素解析を行います。Pythonはプログラミング言語の一
単語 'Python' のコンテキスト: 文章です。形態素解析を行います。Pythonはプログラミング言語の一つです。
単語 'グゲ' のコンテキスト: 。形態素解析を行います。Pythonはプログラミング言語の一つです。

check_hitメソッドの出力:
単語 'テスト' のヒット情報: {'hit': False, 'surfac

1it [00:00, 3908.95it/s]

                                         surface  \
0  これはテストの文章です。形態素解析を行います。Pythonはプログラミング言語の一つです。   

                                      surface_tokens  \
0  [これ, は, テスト, の, 文章, です, 。, 形態, 素, 解析, を, 行い, ま...   

                                pronunciation_tokens  \
0  [コレ, ワ, テスト, ノ, ブンショー, デス, , ケータイ, ソ, カイセキ, オ,...   

                                       pronunciation  \
0  コレワテストノブンショーデスケータイソカイセキオオコナイマスPythonワプログラミングゲン...   

                                             hitword  
0  {'hitwords': [{'hit': True, 'surface': 'グゲ', '...  


In [175]:
# テストデータの作成
test_data = {
    'curid': [1, 2],
    'title': ['テストタイトル1', 'テストタイトル2'],
    'hitword': [
        {
            "hitwords": [
                {'hit': True, 'surface': 'テスト', 'pronunciation': 'テスト', 'length': 3, 'is_crossing_word_boundary': False, 'context': 'これはテストの文章です。', 'matched_passage_surface': 'テスト'},
                {'hit': True, 'surface': '解析', 'pronunciation': 'カイセキ', 'length': 4, 'is_crossing_word_boundary': True, 'context': '形態素解析を行います。', 'matched_passage_surface': '解析'}
            ],
            "sum_length": 7,
            "max_length": 4,
            "num": 2
        },
        {
            "hitwords": [
                {'hit': True, 'surface': 'Python', 'pronunciation': 'パイソン', 'length': 5, 'is_crossing_word_boundary': False, 'context': 'Pythonはプログラミング言語の一つです。', 'matched_passage_surface': 'Python'}
            ],
            "sum_length": 5,
            "max_length": 5,
            "num": 1
        }
    ]
}

# データフレームの作成
passage_df = pd.DataFrame(test_data)

# SerendipitousWordDetectorクラスのインスタンスを作成
detector = SerendipitousWordDetector()

# convert_tableメソッドをテスト
converted_df = detector.convert_table(passage_df)

# 結果の表示
print(converted_df)

2it [00:00, 3300.00it/s]

   curid     title word_surface word_pronunciation  word_length  \
0      1  テストタイトル1          テスト                テスト            3   
1      1  テストタイトル1           解析               カイセキ            4   
2      2  テストタイトル2       Python               パイソン            5   

   is_crossing_word_boundary                 context matched_passage_surface  \
0                      False            これはテストの文章です。                     テスト   
1                       True             形態素解析を行います。                      解析   
2                      False  Pythonはプログラミング言語の一つです。                  Python   

   sum_length  max_length  num  
0           7           4    2  
1           7           4    2  
2           5           5    1  


# 単語リストのダウンロードと整形

単語リストをダウンロードし、data/poke.csvというパスに保存します。

```sh
# macの場合
curl -o data/poke.txt https://wonderhorn.net/material/poke.txt
```

poke.csvの中身は以下のような感じです。shiftjisでencodeされていることに注意してください。

```data/poke.txt
フシギダネ
フシギソウ
フシギバナ
ヒトカゲ
...
```

surfaceという列名をもつcsvとして、utf8で保存し直します。もしくは手動で書き換え、再エンコードを行っても良いと思います。

In [146]:
def format_poketext(input_path: str, output_path: str
                    , *
                    , input_encoding = "shiftjis"
                    , output_encoding = "utf8")->None:
    df = pd.read_csv(input_path, header=None, encoding=input_encoding)
    df.columns=["surface"]
    df.to_csv(output_path, encoding=output_encoding, index=False)
  
RAW_FILE_PATH = "data/poke.txt"
WORDLIST_PATH = "data/poke_formatted.csv"

format_poketext(RAW_FILE_PATH, WORDLIST_PATH)    

In [12]:
from datasets import load_dataset

# URL of the dataset
dataset_url = "izumi-lab/wikipedia-ja-20230720"

# Load the dataset
dataset = load_dataset(dataset_url)

# To view basic information or manipulate the dataset, you can use:
print(dataset)


DatasetDict({
    train: Dataset({
        features: ['curid', 'title', 'text'],
        num_rows: 1362415
    })
})


In [120]:
subset = dataset["train"][:3]
passage_df = pd.DataFrame(subset)
passage_df.rename(columns={"text": "surface"}, inplace=True)
print(passage_df.head())

  curid   title                                            surface
0     5  アンパサンド  アンパサンド（&amp;, ）は、並立助詞「…と…」を意味する記号である。ラテン語で「…と…...
1    10      言語  言語（げんご、language）は、狭義には「声による記号の体系」をいう。\n広辞苑や大辞泉...
2    11     日本語  日本語（にほんご、にっぽんご、）は、日本国内や、かつての日本領だった国、そして国外移民や移住...


In [178]:
def split_surface_to_sentences(df: pd.DataFrame) -> pd.DataFrame:
    """
    DataFrame内の'surface'列に含まれるテキストを文に分割し、新しいDataFrameを作成する関数。
    各行は'surface'列のテキストを句点「。」で分割し、それぞれの文を新しい行として追加する。
    分割された文は'surface'列に格納される。
    df: 分割する文が含まれるDataFrame。'surface'列が必要。
    戻り値: 'surface'列に分割された文を含む新しいDataFrame。
    """
    def split_sentences(row):
        # 'surface'列のテキストを行ごとに分割し、さらに句点「。」で文に分割する
        sentences = [sentence for line in row['surface'].splitlines() for sentence in line.split('。') if sentence]
        # 分割された文を新しいDataFrameに格納し、他の列は元の値を繰り返す
        return pd.DataFrame({col: [row[col]]*len(sentences) if col != 'surface' else sentences for col in df.columns})

    # 全ての行に対してsplit_sentences関数を適用し、結果を結合する
    new_df = pd.concat(df.apply(split_sentences, axis=1).tolist(), ignore_index=True)
    return new_df

subset = dataset["train"][:3]
article_df = pd.DataFrame(subset)
article_df.rename(columns={"text": "surface"}, inplace=True)
passage_df = split_surface_to_sentences(article_df)
print(len(article_df))
print(article_df.head())

3
  curid   title                                            surface
0     5  アンパサンド  アンパサンド（&amp;, ）は、並立助詞「…と…」を意味する記号である。ラテン語で「…と…...
1    10      言語  言語（げんご、language）は、狭義には「声による記号の体系」をいう。\n広辞苑や大辞泉...
2    11     日本語  日本語（にほんご、にっぽんご、）は、日本国内や、かつての日本領だった国、そして国外移民や移住...


In [181]:
detector = SerendipitousWordDetector()
word_df = pd.read_csv(WORDLIST_PATH)
# check_hitsメソッドをテスト
hitwords_df = detector.check_hits(df, passage_df)

converted_df = detector.convert_table(hitwords_df)

hitwords_df.to_csv(f"data/poke_hitwords_{len(article_df)}.csv", index=False)
converted_df.to_csv(f"data/poke_hitwords_{len(article_df)}_converted.csv", index=False)

1589it [00:01, 1329.87it/s]


     curid title                                            surface  \
59      10    言語  言語と非言語の境界が問題になるが、文字を使う方法と文字を用いない方法の区別のみで、言語表現を...   
106     10    言語      同一語族に属する言語群の場合、共通語彙から言語の分化した年代を割り出す方法も考案されている   
194     10    言語  学校教育もこの言語で行われるが、民族語とかけ離れた存在であることもあり国民の中で使用できる層...   
297     11   日本語  大野晋は日本語が語彙・文法などの点でタミル語と共通点を持つとの説を唱えるが、比較言語学の方法...   
336     11   日本語         これは、「直前の母音を1モーラ分引く」という方法で発音される独立した特殊モーラである   
345     11   日本語  たとえば、「です」「ます」は のように発音されるし、「菊」「力」「深い」「放つ」「秋」などは...   
369     11   日本語  なお、「だ行」の「ぢ」「づ」は、一部方言を除いて「ざ行」の「じ」「ず」と同音に帰しており、発...   
538     11   日本語                                       」「姉さん、どこへ行くの   
578     11   日本語        英訳で "We hold…"（われらは信ずる）と主語・述語が隣り合うのとは対照的である   
614     11   日本語  「気を引かれる」「私は泣かない」「花が笑った」「さあ、出かけよう」「今日は来ないそうだ」「も...   
634     11   日本語  なお、上記はあくまでも典型的な機能であり、主体を表さない「が」（例、「水が飲みたい」）、対象...   
718     11   日本語  漫画家の手塚治虫は、漫画を英訳してもらったところ、「ドギューン」「シーン」などの語に翻訳者が...   
752     11   日本語                      この説明方法は、現在の学校教育の国語でも取り入れられている   
808   

26it [00:00, 25366.81it/s]


   curid title word_surface word_pronunciation  word_length  \
0     10    言語         ホーホー                ホーホ            3   
2     10    言語          ドオー                ドオー            3   
5     11   日本語         デスマス               デスマス            4   
7     11   日本語          サンド                サンド            3   
8     11   日本語          ヤクデ                ヤクデ            3   
9     11   日本語          ラッタ                ラッタ            3   
10    11   日本語         ドードー               ドードー            4   
11    11   日本語          ラッタ                ラッタ            3   
13    11   日本語          ゴース                ゴース            3   
14    11   日本語          ドオー                ドオー            3   
15    11   日本語          フシデ                フシデ            3   
16    11   日本語         ホーホー                ホーホ            3   
17    11   日本語         デスマス               デスマス            4   
18    11   日本語          メテノ                メテノ            3   
20    11   日本語         ホーホー                ホーホ         